# Telegram Meteora Pool Analyzer Report Generator

## 📊 Overview
This notebook generates automated liquidity pool analysis reports using Gateway's Meteora CLMM endpoints and distributes them via Telegram. It fetches real-time pool data, analyzes liquidity distribution, price movements, and generates comprehensive reports with charts.

## 📋 Prerequisites
- Gateway service running locally (default: http://localhost:15888)
- Telegram bot configuration (bot token and chat ID in .env)

## 📈 Expected Outputs
- Real-time pool metrics and liquidity analysis
- Liquidity distribution charts showing bin concentrations
- CSV reports with detailed bin data

In [208]:
# 🔌 Initialize Core Services and Libraries

## ⚙️ Configuration and Gateway API Client

import warnings
warnings.filterwarnings("ignore")

import sys
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from typing import Dict, List, Optional, Any
import json

# Add the project root to the path to ensure imports work
project_root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import QuantsLab Gateway Data Source (our new utility class)
try:
    from core.data_sources.gateway import GatewayDataSource
    print("✅ GatewayDataSource imported successfully")
    FALLBACK_MODE = False
except ImportError as e:
    print(f"❌ Failed to import GatewayDataSource: {e}")
    print("💡 Falling back to direct Hummingbot imports...")
    FALLBACK_MODE = True
    
    # Fallback to direct imports
    from hummingbot.core.gateway.gateway_http_client import GatewayHttpClient
    from hummingbot.client.config.client_config_map import GatewayConfigMap
    
    # Create a minimal fallback class with essential methods
    class GatewayDataSource:
        def __init__(self, gateway_url: str, network: str):
            self.gateway_url = gateway_url
            self.network = network
            self.gateway_config = GatewayConfigMap()
            self.gateway_config.gateway_api_host = gateway_url.replace('http://', '').replace('https://', '').split(':')[0]
            self.gateway_config.gateway_api_port = str(gateway_url.split(':')[-1])
            self.gateway_config.gateway_use_ssl = False
            self.client = GatewayHttpClient.get_instance(self.gateway_config)
            self.token_cache = {}
        
        async def ping_gateway(self):
            return await self.client.ping_gateway()
        
        async def fetch_token_info(self, token_address: str, network: Optional[str] = None) -> Optional[Dict]:
            """Fetch token information from Gateway with caching"""
            network = network or self.network
            cache_key = f"{token_address}_{network}"
            
            if cache_key in self.token_cache:
                return self.token_cache[cache_key]
                
            try:
                response = await self.client.api_request(
                    method="get",
                    path_url=f"tokens/{token_address}",
                    params={
                        'chain': 'solana',
                        'network': network
                    }
                )
                token_info = response.get('token', {})
                self.token_cache[cache_key] = token_info
                return token_info
            except Exception as e:
                print(f"❌ Error fetching token info for {token_address}: {e}")
                self.token_cache[cache_key] = None
                return None
        
        def calculate_price_range_distribution(self, bins_data, current_price: float, price_col='price', liquidity_col='total_value', ranges=[5, 10, 15, 20]):
            """Calculate liquidity distribution at different price ranges"""
            total_liquidity = bins_data[liquidity_col].sum()
            if total_liquidity == 0:
                return {pct: 0.0 for pct in ranges}
            
            distributions = {}
            for pct in ranges:
                lower_price = current_price * (1 - pct/100)
                upper_price = current_price * (1 + pct/100)
                
                bins_in_range = bins_data[
                    (bins_data[price_col] >= lower_price) &
                    (bins_data[price_col] <= upper_price)
                ]
                
                liquidity_in_range = bins_in_range[liquidity_col].sum() if not bins_in_range.empty else 0
                pct_of_total = (liquidity_in_range / total_liquidity * 100) if total_liquidity > 0 else 0
                
                distributions[pct] = pct_of_total
            
            return distributions
        
        def create_liquidity_chart(self, bins_data, price_col='price', liquidity_col='total_value', bin_id_col='binId', current_price=None, active_bin_id=None, title="Liquidity Distribution"):
            """Create a basic liquidity chart"""
            if bins_data.empty:
                return None
                
            fig = go.Figure()
            
            # Color bins based on active bin
            colors = ['red' if (active_bin_id and bid < active_bin_id)
                     else 'green' if (active_bin_id and bid > active_bin_id)
                     else 'yellow' for bid in bins_data[bin_id_col]]
            
            fig.add_trace(
                go.Bar(
                    x=bins_data[price_col],
                    y=bins_data[liquidity_col],
                    name='Liquidity',
                    marker=dict(color=colors, opacity=0.7),
                    hovertemplate=(
                        f'Price: %{{x:.8f}}<br>'
                        f'Liquidity: %{{y:,.0f}}<br>'
                        f'Bin ID: %{{customdata}}<br>'
                        '<extra></extra>'
                    ),
                    customdata=bins_data[bin_id_col]
                )
            )
            
            if current_price:
                fig.add_vline(x=current_price, line_dash="dash", line_color="blue", line_width=2,
                             annotation_text=f"Current Price: {self.format_price(current_price)}")
            
            fig.update_xaxes(title_text="Price", showgrid=True)
            fig.update_yaxes(title_text="Liquidity")
            fig.update_layout(title=f"{title}<br><sub>{self.format_timestamp(datetime.now(timezone.utc))}</sub>",
                            height=600, width=1200, template="plotly_white")
            
            return fig
        
        def save_chart(self, fig, filename: str, format: str = "png") -> bool:
            """Save chart to file"""
            try:
                if format.lower() == "png":
                    fig.write_image(filename, width=1200, height=800, scale=2)
                elif format.lower() == "html":
                    fig.write_html(filename)
                return True
            except ImportError:
                # Fallback to HTML if kaleido not available
                html_filename = filename.replace('.png', '.html')
                fig.write_html(html_filename)
                return True
            except Exception as e:
                print(f"❌ Error saving chart: {e}")
                return False
        
        def export_to_csv(self, data, filename: str, metadata=None) -> bool:
            """Export DataFrame to CSV"""
            try:
                if metadata:
                    for key, value in metadata.items():
                        if key not in data.columns:
                            data[key] = value
                data['export_timestamp'] = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')
                data.to_csv(filename, index=False, float_format='%.8f')
                return True
            except Exception as e:
                print(f"❌ Error exporting CSV: {e}")
                return False
        
        @staticmethod
        def format_number(value, decimals=2, is_currency=False):
            if is_currency:
                if value >= 1_000_000: return f"${value/1_000_000:.{decimals}f}M"
                elif value >= 1_000: return f"${value/1_000:.{decimals}f}K"
                else: return f"${value:.{decimals}f}"
            else:
                if value >= 1_000_000: return f"{value/1_000_000:.{decimals}f}M"
                elif value >= 1_000: return f"{value/1_000:.{decimals}f}K"
                else: return f"{value:.{decimals}f}"
        
        @staticmethod
        def format_price(price):
            if price >= 100: return f"${price:.2f}"
            elif price >= 10: return f"${price:.3f}"
            elif price >= 1: return f"${price:.4f}"
            elif price >= 0.1: return f"${price:.5f}"
            elif price >= 0.01: return f"${price:.6f}"
            elif price >= 0.001: return f"${price:.7f}"
            elif price >= 0.0001: return f"${price:.8f}"
            elif price > 0:
                if price < 0.00000001: return f"${price:.2e}"
                else: return f"${price:.10f}"
            else: return "$0.00"
        
        @staticmethod
        def format_percentage(value, decimals=2):
            return f"{value:.{decimals}f}%"
        
        @staticmethod
        def format_timestamp(timestamp, format_str='%Y-%m-%d %H:%M UTC'):
            if timestamp.tzinfo is None:
                timestamp = timestamp.replace(tzinfo=timezone.utc)
            return timestamp.strftime(format_str)
        
        def get_cache_stats(self):
            cached_tokens = len([k for k, v in self.token_cache.items() if v is not None])
            failed_lookups = len([k for k, v in self.token_cache.items() if v is None])
            return {
                'token_cache_size': len(self.token_cache),
                'cached_tokens': cached_tokens,
                'failed_lookups': failed_lookups
            }

# Import QuantsLab notification system
from core.notifiers import NotificationManager, NotificationMessage

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully")
if FALLBACK_MODE:
    print("📊 Services available: GatewayDataSource (Fallback mode), Notification system")
else:
    print("📊 Services available: GatewayDataSource (QuantsLab), Notification system")
print("⚙️ Environment variables loaded from .env")

✅ GatewayDataSource imported successfully
✅ Libraries imported successfully
📊 Services available: GatewayDataSource (QuantsLab), Notification system
⚙️ Environment variables loaded from .env


In [209]:
# Configuration Parameters and Gateway Setup
GATEWAY_URL = os.getenv('GATEWAY_URL', 'http://localhost:15888')
NETWORK = 'mainnet-beta'
UPDATE_INTERVAL_SECONDS = 300  # 5 minutes between updates

# Pool addresses to analyze - add your pool addresses here
POOL_ADDRESSES = [
    '2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3',  # SOL-USDC
    'C8Gr6AUuq9hEdSYJzoEpNcdjpojPZwqG5MtQbeouNNwg',  # JUP-SOL
    '9d9mb8kooFfaD3SctgZtkxQypkshx6ezhbKio89ixyy2',  # TRUMP-USDC
    # Add more pool addresses here as needed
    # 'your_pool_address_here',
]

# Initialize Gateway Data Source
gateway_ds = GatewayDataSource(gateway_url=GATEWAY_URL, network=NETWORK)

# Initialize notification manager
notification_manager = NotificationManager()
enabled_notifiers = notification_manager.get_enabled_notifiers()

# Initialize collections for Telegram sending
telegram_reports = {
    'text_reports': [],
    'chart_files': [],
    'csv_files': [],
    'report_message': None
}

# 📊 Meteora Pool Analyzer Class
class MeteorPoolAnalyzer:
    """Helper class for fetching and analyzing Meteora pools using GatewayDataSource"""
    
    def __init__(self, gateway_ds: GatewayDataSource):
        self.gateway_ds = gateway_ds
        self.client = gateway_ds.client
        
    async def fetch_pool_info(self, pool_address: str, network: str = 'mainnet-beta') -> Optional[Dict]:
        """Fetch Meteora pool information from Gateway"""
        try:
            # Make request to Gateway's Meteora endpoint
            response = await self.client.api_request(
                method="get",
                path_url="connectors/meteora/clmm/pool-info",
                params={
                    'network': network,
                    'poolAddress': pool_address
                }
            )
            return response
        except Exception as e:
            print(f"❌ Error fetching pool info: {e}")
            return None
    
    async def fetch_token_info(self, token_address: str, network: str = 'mainnet-beta') -> Optional[Dict]:
        """Fetch token information from Gateway with caching"""
        return await self.gateway_ds.fetch_token_info(token_address, network)
    
    def calculate_liquidity_distribution(self, bins_df, current_price: float, total_liquidity: float) -> Dict[str, float]:
        """Calculate liquidity distribution at different price ranges"""
        return self.gateway_ds.calculate_price_range_distribution(
            bins_df, current_price, 'price', 'total_value'
        )
    
    def analyze_pool(self, pool_data: Dict, metadata: Dict = None) -> Dict:
        """Analyze pool data and calculate metrics"""
        # Add metadata if provided
        if metadata:
            pool_data.update(metadata)
            
        # Calculate total liquidity
        price = pool_data['price']
        base_amount = pool_data['baseTokenAmount']
        quote_amount = pool_data['quoteTokenAmount']
        total_liquidity_usd = quote_amount + (base_amount * price)
        
        analysis = {
            'address': pool_data['address'],
            'pool_name': pool_data.get('pool_name', 'Unknown'),
            'base_token': pool_data.get('base_token_symbol', pool_data.get('base_token', 'BASE')),
            'quote_token': pool_data.get('quote_token_symbol', pool_data.get('quote_token', 'QUOTE')),
            'base_token_address': pool_data.get('baseTokenAddress'),
            'quote_token_address': pool_data.get('quoteTokenAddress'),
            'current_price': price,
            'fee_pct': pool_data['feePct'],
            'dynamic_fee_pct': pool_data.get('dynamicFeePct', pool_data['feePct']),
            'bin_step': pool_data.get('binStep', 0),
            'active_bin_id': pool_data.get('activeBinId'),
            'base_liquidity': base_amount,
            'quote_liquidity': quote_amount,
            'total_liquidity_usd': total_liquidity_usd,
            'num_bins': len(pool_data.get('bins', [])),
            'liquidity_concentration': 0,
            'liquidity_distribution': {},
            'price_range': {'min': 0, 'max': 0},
            'timestamp': datetime.now(timezone.utc)
        }
        
        # Analyze bins if available
        bins = pool_data.get('bins', [])
        if bins:
            bins_df = pd.DataFrame(bins)
            
            # Calculate price range
            analysis['price_range']['min'] = bins_df['price'].min()
            analysis['price_range']['max'] = bins_df['price'].max()
            
            # Calculate bin values
            bins_df['total_value'] = (
                bins_df['quoteTokenAmount'] + 
                bins_df['baseTokenAmount'] * price
            )
            
            total_liq = bins_df['total_value'].sum()
            
            # Calculate concentration around active bin
            if analysis['active_bin_id'] is not None:
                active_bin_id = analysis['active_bin_id']
                nearby_bins = bins_df[
                    (bins_df['binId'] >= active_bin_id - 5) &
                    (bins_df['binId'] <= active_bin_id + 5)
                ]
                
                nearby_liq = nearby_bins['total_value'].sum() if not nearby_bins.empty else 0
                
                if total_liq > 0:
                    analysis['liquidity_concentration'] = (nearby_liq / total_liq) * 100
            
            # Calculate liquidity distribution at different price ranges using GatewayDataSource
            analysis['liquidity_distribution'] = self.calculate_liquidity_distribution(
                bins_df, price, total_liq
            )
            
            # Find key liquidity levels
            top_bins = bins_df.nlargest(5, 'total_value')
            analysis['key_levels'] = [
                {'price': row['price'], 'total_value': row['total_value']}
                for _, row in top_bins.iterrows()
            ]
            
            analysis['bins_df'] = bins_df
            
        return analysis

# Initialize the analyzer
pool_analyzer = MeteorPoolAnalyzer(gateway_ds)

async def generate_pool_report(analyses: List[Dict]) -> str:
    """Generate formatted pool analysis report for Telegram using cached token symbols"""
    
    if not analyses:
        return "⚠️ No pool data available for analysis"
    
    # Generate report timestamp
    report_time = gateway_ds.format_timestamp(datetime.now(timezone.utc))
    
    # Build the report message
    report_message = f"""🏊 <b>Meteora Pool Analysis Report</b>
📅 {report_time}
🔗 Network: Solana {NETWORK}
"""
    
    # Add analysis for each pool
    for analysis in analyses:
        # Use cached token symbols from initial pool fetching
        base_symbol = analysis['base_token']
        quote_symbol = analysis['quote_token']
        
        # Only try to fetch if we don't already have symbols and they're not addresses
        if analysis.get('base_token_address') and len(base_symbol) > 10:  # Likely an address, not symbol
            cache_key = f"{analysis['base_token_address']}_{NETWORK}"
            if cache_key in gateway_ds.token_cache:
                cached_info = gateway_ds.token_cache[cache_key]
                if cached_info and 'symbol' in cached_info:
                    base_symbol = cached_info['symbol']
                    
        if analysis.get('quote_token_address') and len(quote_symbol) > 10:  # Likely an address, not symbol
            cache_key = f"{analysis['quote_token_address']}_{NETWORK}"
            if cache_key in gateway_ds.token_cache:
                cached_info = gateway_ds.token_cache[cache_key]
                if cached_info and 'symbol' in cached_info:
                    quote_symbol = cached_info['symbol']
        
        # Create token pair name with fee tier and bin step
        token_pair = f"{base_symbol}/{quote_symbol}"
        fee_tier = f"{analysis['fee_pct']:.2f}%"
        bin_step = analysis['bin_step']
        
        # Pool header with token pair, fee tier, and bin step
        report_message += f"""📊 <b>{token_pair} • {fee_tier} • Bin:{bin_step}</b>
<i>Pool: {analysis['pool_name']}</i>
━━━━━━━━━━━━━━━━━━━━
"""
        
        # Price and liquidity info with smart formatting using GatewayDataSource
        price = analysis['current_price']
        price_str = gateway_ds.format_price(price)
        report_message += f"""💰 <b>Price & Liquidity</b>
• Current Price: <code>{price_str}</code>
• Total Liquidity: <code>{gateway_ds.format_number(analysis['total_liquidity_usd'], is_currency=True)}</code>
• {base_symbol}: <code>{gateway_ds.format_number(analysis['base_liquidity'], decimals=2)}</code>
• {quote_symbol}: <code>{gateway_ds.format_number(analysis['quote_liquidity'], decimals=2)}</code>

"""
        
        # Pool parameters
        report_message += f"""⚙️ <b>Pool Parameters</b>
• Fee: <code>{gateway_ds.format_percentage(analysis['fee_pct'])}</code>
• Bin Step: <code>{analysis['bin_step']}</code>
• Active Bin: <code>{analysis['active_bin_id']}</code>
• Total Bins: <code>{analysis['num_bins']}</code>

"""
        
        # Liquidity distribution with smart price formatting
        min_price_str = gateway_ds.format_price(analysis['price_range']['min'])
        max_price_str = gateway_ds.format_price(analysis['price_range']['max'])
        report_message += f"""📈 <b>Liquidity Distribution</b>
• Price Range: <code>{min_price_str} - {max_price_str}</code>
"""
        
        # Show liquidity at different ranges instead of key levels
        if analysis.get('liquidity_distribution'):
            report_message += f"\n🎯 <b>Liquidity Around Price</b>\n"
            for pct in [5, 10, 15, 20]:
                if pct in analysis['liquidity_distribution']:
                    liquidity_pct = analysis['liquidity_distribution'][pct]
                    report_message += f"• ±{pct}%: <code>{liquidity_pct:.1f}%</code> of total liquidity\n"
        
        report_message += "\n"
        
    return report_message

def create_liquidity_chart(analysis: Dict) -> go.Figure:
    """Create liquidity distribution chart for a pool using GatewayDataSource"""
    
    if 'bins_df' not in analysis or analysis['bins_df'].empty:
        return None
    
    bins_df = analysis['bins_df']
    
    # Use GatewayDataSource chart creation with enhanced title
    base_token = analysis.get('base_token', 'BASE')
    quote_token = analysis.get('quote_token', 'QUOTE')
    fee_tier = f"{analysis['fee_pct']:.2f}%"
    bin_step = analysis['bin_step']
    
    title = f"{base_token}/{quote_token} • {fee_tier} Fee • Bin Step: {bin_step}"
    
    fig = gateway_ds.create_liquidity_chart(
        bins_data=bins_df,
        price_col='price',
        liquidity_col='total_value',
        bin_id_col='binId',
        current_price=analysis['current_price'],
        active_bin_id=analysis['active_bin_id'],
        title=title
    )
    
    return fig

print(f"📊 Configuration loaded:")
print(f"  - Gateway URL: {GATEWAY_URL}")
print(f"  - Network: {NETWORK}")
print(f"  - Update Interval: {UPDATE_INTERVAL_SECONDS}s")
print(f"  - Pool Addresses: {len(POOL_ADDRESSES)} configured")
for i, addr in enumerate(POOL_ADDRESSES, 1):
    print(f"    {i}. {addr}")
print(f"🔔 Enabled notifiers: {', '.join(enabled_notifiers) if enabled_notifiers else 'None configured'}")
print(f"✅ GatewayDataSource initialized with Gateway HTTP Client")
print("✅ Meteora Pool Analyzer initialized with GatewayDataSource")
print("📤 Telegram reports will be collected and sent at the end")
print("🗂️ Token cache initialized for efficient lookups")
print(f"📈 Cache stats: {gateway_ds.get_cache_stats()}")

📊 Configuration loaded:
  - Gateway URL: http://localhost:15888
  - Network: mainnet-beta
  - Update Interval: 300s
  - Pool Addresses: 3 configured
    1. 2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3
    2. C8Gr6AUuq9hEdSYJzoEpNcdjpojPZwqG5MtQbeouNNwg
    3. 9d9mb8kooFfaD3SctgZtkxQypkshx6ezhbKio89ixyy2
🔔 Enabled notifiers: telegram
✅ GatewayDataSource initialized with Gateway HTTP Client
✅ Meteora Pool Analyzer initialized with GatewayDataSource
📤 Telegram reports will be collected and sent at the end
🗂️ Token cache initialized for efficient lookups
📈 Cache stats: {'token_cache_size': 0, 'cached_tokens': 0, 'failed_lookups': 0}


In [210]:
# 📡 Fetch Pool Data and Token Metadata from Gateway
print(f"🔄 Fetching pool data from Gateway...")
print(f"📍 Gateway URL: {GATEWAY_URL}")
print(f"🔗 Network: {NETWORK}")
print(f"🏊 Pools to analyze: {len(POOL_ADDRESSES)}\n")

# Test gateway connection using GatewayDataSource
try:
    is_online = await gateway_ds.ping_gateway()
    if is_online:
        print("✅ Gateway connection successful\n")
    else:
        print("⚠️ Gateway is offline, trying to connect...\n")
except Exception as e:
    print(f"❌ Gateway connection error: {e}\n")
    is_online = False

# Fetch all pool data if online
all_pool_analyses = []
if is_online:
    print("🔄 Fetching data for all pools with token metadata...")
    
    for i, pool_address in enumerate(POOL_ADDRESSES, 1):
        print(f"\n📊 Pool {i}/{len(POOL_ADDRESSES)}: {pool_address}")
        
        try:
            # Fetch pool data
            pool_data = await pool_analyzer.fetch_pool_info(pool_address, NETWORK)
            
            if pool_data:
                print(f"✅ Raw pool data fetched")
                
                # Display raw pool info
                print(f"🏊 RAW POOL DATA:")
                print(f"  Pool Address: {pool_data['address']}")
                print(f"  Base Token Address: {pool_data['baseTokenAddress']}")
                print(f"  Quote Token Address: {pool_data['quoteTokenAddress']}")
                print(f"  Current Price: {gateway_ds.format_price(pool_data['price'])}")
                print(f"  Fee: {gateway_ds.format_percentage(pool_data['feePct'])}")
                print(f"  Bin Step: {pool_data['binStep']}")
                print(f"  Active Bin ID: {pool_data['activeBinId']}")
                print(f"  Total Bins: {len(pool_data.get('bins', []))}")
                
                # Fetch token metadata using GatewayDataSource
                print(f"🔍 Looking up token metadata...")
                base_token_info = None
                quote_token_info = None
                base_symbol = pool_data['baseTokenAddress']  # Use address as fallback
                quote_symbol = pool_data['quoteTokenAddress']  # Use address as fallback
                
                # Lookup base token
                if pool_data.get('baseTokenAddress'):
                    try:
                        base_token_info = await pool_analyzer.fetch_token_info(pool_data['baseTokenAddress'], NETWORK)
                        if base_token_info and 'symbol' in base_token_info:
                            base_symbol = base_token_info['symbol']
                            print(f"  ✅ Base Token: {base_token_info['name']} ({base_token_info['symbol']})")
                            print(f"    Address: {base_token_info['address']}")
                            print(f"    Decimals: {base_token_info['decimals']}")
                        else:
                            print(f"  ⚠️ Base Token: Could not fetch metadata")
                    except Exception as e:
                        print(f"  ❌ Base Token lookup error: {e}")
                
                # Lookup quote token  
                if pool_data.get('quoteTokenAddress'):
                    try:
                        quote_token_info = await pool_analyzer.fetch_token_info(pool_data['quoteTokenAddress'], NETWORK)
                        if quote_token_info and 'symbol' in quote_token_info:
                            quote_symbol = quote_token_info['symbol']
                            print(f"  ✅ Quote Token: {quote_token_info['name']} ({quote_token_info['symbol']})")
                            print(f"    Address: {quote_token_info['address']}")
                            print(f"    Decimals: {quote_token_info['decimals']}")
                        else:
                            print(f"  ⚠️ Quote Token: Could not fetch metadata")
                    except Exception as e:
                        print(f"  ❌ Quote Token lookup error: {e}")
                
                # Create token pair name
                token_pair = f"{base_symbol}/{quote_symbol}"
                pool_name = f"{token_pair} Pool"
                
                print(f"🏷️ Identified Pool: {token_pair}")
                
                # Create metadata with token info
                metadata = {
                    'pool_name': pool_name,
                    'token_pair': token_pair,
                    'base_token_symbol': base_symbol,
                    'quote_token_symbol': quote_symbol,
                    'base_token_info': base_token_info,
                    'quote_token_info': quote_token_info
                }
                
                # Perform analysis
                analysis = pool_analyzer.analyze_pool(pool_data, metadata)
                all_pool_analyses.append(analysis)
                
                print(f"📊 Analysis Summary:")
                print(f"  Price: {gateway_ds.format_price(analysis['current_price'])}")
                print(f"  Liquidity: {gateway_ds.format_number(analysis['total_liquidity_usd'], is_currency=True)}")
                print(f"  Base ({base_symbol}): {gateway_ds.format_number(analysis['base_liquidity'], decimals=2)}")
                print(f"  Quote ({quote_symbol}): {gateway_ds.format_number(analysis['quote_liquidity'], decimals=2)}")
                print(f"  Bins: {analysis['num_bins']}")
                
            else:
                print(f"❌ Failed to fetch data for pool {pool_address}")
                
        except Exception as e:
            print(f"❌ Error analyzing pool {pool_address}: {e}")
    
    print(f"\n✅ Successfully analyzed {len(all_pool_analyses)} out of {len(POOL_ADDRESSES)} pools")
    
    if all_pool_analyses:
        total_liquidity = sum(a['total_liquidity_usd'] for a in all_pool_analyses)
        print(f"💰 Total Liquidity Across All Pools: {gateway_ds.format_number(total_liquidity, is_currency=True)}")
        
        # Display summary with token pairs using GatewayDataSource formatting
        print(f"\n📊 POOL ANALYSIS SUMMARY:")
        print("=" * 80)
        for analysis in all_pool_analyses:
            token_pair = analysis.get('token_pair', analysis['pool_name'])
            print(f"{token_pair:<20} | "
                  f"{gateway_ds.format_price(analysis['current_price']):<12} | "
                  f"{gateway_ds.format_number(analysis['total_liquidity_usd'], is_currency=True):<12} | "
                  f"{analysis['num_bins']:>4} bins")
        print("=" * 80)
        
        # Display cache statistics
        cache_stats = gateway_ds.get_cache_stats()
        print(f"\n🗂️ Token Cache Stats: {cache_stats['cached_tokens']} tokens cached, {cache_stats['failed_lookups']} failed lookups")
        
else:
    print("❌ Cannot fetch pool data - Gateway is offline")
    all_pool_analyses = []

🔄 Fetching pool data from Gateway...
📍 Gateway URL: http://localhost:15888
🔗 Network: mainnet-beta
🏊 Pools to analyze: 3

✅ Gateway connection successful

🔄 Fetching data for all pools with token metadata...

📊 Pool 1/3: 2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3
✅ Raw pool data fetched
🏊 RAW POOL DATA:
  Pool Address: 2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3
  Base Token Address: So11111111111111111111111111111111111111112
  Quote Token Address: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
  Current Price: $209.67
  Fee: 0.04%
  Bin Step: 100
  Active Bin ID: -157
  Total Bins: 141
🔍 Looking up token metadata...
  ✅ Base Token: Wrapped Solana (SOL)
    Address: So11111111111111111111111111111111111111112
    Decimals: 9
  ✅ Quote Token: USDC (USDC)
    Address: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
    Decimals: 6
🏷️ Identified Pool: SOL/USDC
📊 Analysis Summary:
  Price: $209.67
  Liquidity: $64.12K
  Base (SOL): 153.82
  Quote (USDC): 31.87K
  Bins: 141

📊 Pool 2/3: C8Gr

2025-09-23 20:47:42,585 - hummingbot.core.gateway.gateway_http_client - WARNING - Call to http://localhost:15888/tokens/6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN failed. See logs for more details.
2025-09-23 20:47:42,585 - core.data_sources.gateway - ERROR - Error fetching token info for 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN: Error on GET http://localhost:15888/tokens/6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN Error: NotFoundError


✅ Raw pool data fetched
🏊 RAW POOL DATA:
  Pool Address: 9d9mb8kooFfaD3SctgZtkxQypkshx6ezhbKio89ixyy2
  Base Token Address: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN
  Quote Token Address: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
  Current Price: $7.6135
  Fee: 0.10%
  Bin Step: 50
  Active Bin ID: 407
  Total Bins: 141
🔍 Looking up token metadata...
  ⚠️ Base Token: Could not fetch metadata
  ✅ Quote Token: USDC (USDC)
    Address: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
    Decimals: 6
🏷️ Identified Pool: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC
📊 Analysis Summary:
  Price: $7.6135
  Liquidity: $317.37M
  Base (6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN): 15.46M
  Quote (USDC): 199.64M
  Bins: 141

✅ Successfully analyzed 3 out of 3 pools
💰 Total Liquidity Across All Pools: $317.45M

📊 POOL ANALYSIS SUMMARY:
SOL/USDC Pool        | $209.67      | $64.12K      |  141 bins
JUP/SOL Pool         | $0.0022009   | $14.98K      |  141 bins
6p6xgHyF7AeE6TZkSmFsko444wqoP

In [211]:
# 🚀 Generate Pool Analysis Report with Token Lookup (Preview Only)
if all_pool_analyses:    
    # Generate report with token symbol lookup using GatewayDataSource
    report_message = await generate_pool_report(all_pool_analyses)
    
    # Store the report for sending later
    telegram_reports['report_message'] = report_message
    
    print("📤 Pool analysis report generated successfully")
    print(f"📊 Report includes {len(all_pool_analyses)} pool(s) with token symbols")
    print("📋 Report stored for Telegram sending at the end\n")
    
    # Display detailed analysis for each pool using GatewayDataSource formatting
    print("📊 DETAILED POOL ANALYSIS")
    print("=" * 80)
    for i, analysis in enumerate(all_pool_analyses, 1):
        print(f"\n📊 POOL {i} - {analysis['pool_name']}")
        print("=" * 60)
        print(f"Address: {analysis['address']}")
        print(f"Current Price: {gateway_ds.format_price(analysis['current_price'])}")
        print(f"Total Liquidity (USD): {gateway_ds.format_number(analysis['total_liquidity_usd'], is_currency=True)}")
        print(f"Base Liquidity: {gateway_ds.format_number(analysis['base_liquidity'], decimals=2)} {analysis['base_token']}")
        print(f"Quote Liquidity: {gateway_ds.format_number(analysis['quote_liquidity'], decimals=2)} {analysis['quote_token']}")
        print(f"Liquidity Concentration: {gateway_ds.format_percentage(analysis['liquidity_concentration'], decimals=1)} (within ±5 bins)")
        print(f"Number of Bins: {analysis['num_bins']}")
        print(f"Price Range: {gateway_ds.format_price(analysis['price_range']['min'])} - {gateway_ds.format_price(analysis['price_range']['max'])}")
        print(f"Fee: {gateway_ds.format_percentage(analysis['fee_pct'])}")
        print(f"Bin Step: {analysis['bin_step']}")
        print(f"Active Bin ID: {analysis['active_bin_id']}")
        
        if analysis.get('key_levels'):
            print(f"\n🎯 KEY LIQUIDITY LEVELS:")
            for j, level in enumerate(analysis['key_levels'], 1):
                print(f"  {j}. Price: {gateway_ds.format_price(level['price'])} | Liquidity: {gateway_ds.format_number(level['total_value'], is_currency=True)}")
        
        print("=" * 60)
    
    print("\n📋 TELEGRAM REPORT PREVIEW:")
    print("=" * 50)
    print(report_message.replace('<b>', '**').replace('</b>', '**')
          .replace('<code>', '`').replace('</code>', '`')
          .replace('<i>', '_').replace('</i>', '_'))
    print("=" * 50)
        
else:
    report_message = await generate_pool_report([])
    print("⚠️ No pool data available for report generation")

📤 Pool analysis report generated successfully
📊 Report includes 3 pool(s) with token symbols
📋 Report stored for Telegram sending at the end

📊 DETAILED POOL ANALYSIS

📊 POOL 1 - SOL/USDC Pool
Address: 2sf5NYcY4zUPXUSmG6f66mskb24t5F8S11pC1Nz5nQT3
Current Price: $209.67
Total Liquidity (USD): $64.12K
Base Liquidity: 153.82 SOL
Quote Liquidity: 31.87K USDC
Liquidity Concentration: 10.1% (within ±5 bins)
Number of Bins: 141
Price Range: $106.58 - $429.22
Fee: 0.04%
Bin Step: 100
Active Bin ID: -157

🎯 KEY LIQUIDITY LEVELS:
  1. Price: $224.80 | Liquidity: $807.58
  2. Price: $227.05 | Liquidity: $804.22
  3. Price: $229.32 | Liquidity: $774.30
  4. Price: $231.61 | Liquidity: $771.14
  5. Price: $233.93 | Liquidity: $766.78

📊 POOL 2 - JUP/SOL Pool
Address: C8Gr6AUuq9hEdSYJzoEpNcdjpojPZwqG5MtQbeouNNwg
Current Price: $0.0022009
Total Liquidity (USD): $14.98K
Base Liquidity: 3.81M JUP
Quote Liquidity: 6.60K SOL
Liquidity Concentration: 20.4% (within ±5 bins)
Number of Bins: 141
Price Range:

In [212]:
# 🎨 Generate Liquidity Distribution Charts with Token Symbols (Save for Later)
if all_pool_analyses:
    print("📊 Generating liquidity distribution charts...\n")
    
    # Clear any existing chart files from previous runs
    telegram_reports['chart_files'] = []
    
    for i, analysis in enumerate(all_pool_analyses, 1):
        print(f"🎨 Chart {i}/{len(all_pool_analyses)}: {analysis['pool_name']}")
        
        # Use cached token symbols from initial pool fetching
        base_symbol = analysis['base_token']
        quote_symbol = analysis['quote_token']
        
        # Use GatewayDataSource token cache if available and token symbols are still addresses
        if analysis.get('base_token_address') and len(base_symbol) > 10:
            cache_key = f"{analysis['base_token_address']}_{NETWORK}"
            if cache_key in gateway_ds.token_cache:
                cached_info = gateway_ds.token_cache[cache_key]
                if cached_info and 'symbol' in cached_info:
                    base_symbol = cached_info['symbol']
                    
        if analysis.get('quote_token_address') and len(quote_symbol) > 10:
            cache_key = f"{analysis['quote_token_address']}_{NETWORK}"
            if cache_key in gateway_ds.token_cache:
                cached_info = gateway_ds.token_cache[cache_key]
                if cached_info and 'symbol' in cached_info:
                    quote_symbol = cached_info['symbol']
        
        # Update analysis with proper symbols for chart generation
        analysis_with_symbols = analysis.copy()
        analysis_with_symbols['base_token'] = base_symbol
        analysis_with_symbols['quote_token'] = quote_symbol
        analysis_with_symbols['token_pair'] = f"{base_symbol}/{quote_symbol}"
        
        # Create chart for this pool with proper token symbols using GatewayDataSource
        fig = create_liquidity_chart(analysis_with_symbols)
        
        if fig:
            print(f"  ✅ Chart generated for {analysis_with_symbols['token_pair']}")
            fig.show()
            
            # Save chart as PNG with token pair, fee tier, and bin step in filename
            safe_pair_name = analysis_with_symbols['token_pair'].replace('/', '_')
            fee_tier = f"{analysis['fee_pct']:.2f}".replace('.', 'p')  # 0.04 -> 0p04
            bin_step = analysis['bin_step']
            chart_filename = f"/tmp/meteora_{safe_pair_name}_{fee_tier}pct_bin{bin_step}_chart.png"
            
            # Use GatewayDataSource save_chart method
            if gateway_ds.save_chart(fig, chart_filename, format="png"):
                print(f"  💾 Chart saved using GatewayDataSource: {chart_filename}")
                
                # Store chart info for Telegram sending later with enhanced caption
                chart_info = {
                    'filename': chart_filename,
                    'pool_name': analysis['pool_name'],
                    'token_pair': analysis_with_symbols['token_pair'],
                    'caption': f"""📊 <b>Liquidity Distribution Chart</b>
🏊 Pool: {analysis_with_symbols['token_pair']} • {gateway_ds.format_percentage(analysis['fee_pct'])} Fee • Bin:{analysis['bin_step']}
💰 Total Liquidity: {gateway_ds.format_number(analysis['total_liquidity_usd'], is_currency=True)}
📍 Current Price: {gateway_ds.format_price(analysis['current_price'])}
🎯 Concentration: {gateway_ds.format_percentage(analysis['liquidity_concentration'], decimals=1)} near price
⚙️ Active Bin: {analysis['active_bin_id']} | Total Bins: {analysis['num_bins']}
"""
                }
                telegram_reports['chart_files'].append(chart_info)
                print(f"  📋 Chart stored for Telegram sending")
                        
            else:
                print(f"  ❌ Error saving chart with GatewayDataSource")
        else:
            print(f"  ⚠️ No bin data available for {analysis['pool_name']}")
            
        print()  # Empty line between pools
        
    print(f"✅ Chart generation completed for {len(all_pool_analyses)} pool(s)")
    print(f"📋 {len(telegram_reports['chart_files'])} charts stored for Telegram sending")
else:
    print("⚠️ No pool analyses available for chart generation")

📊 Generating liquidity distribution charts...

🎨 Chart 1/3: SOL/USDC Pool
  ✅ Chart generated for SOL/USDC


  💾 Chart saved using GatewayDataSource: /tmp/meteora_SOL_USDC_0p04pct_bin100_chart.png
  📋 Chart stored for Telegram sending

🎨 Chart 2/3: JUP/SOL Pool
  ✅ Chart generated for JUP/SOL


  💾 Chart saved using GatewayDataSource: /tmp/meteora_JUP_SOL_0p15pct_bin80_chart.png
  📋 Chart stored for Telegram sending

🎨 Chart 3/3: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC Pool
  ✅ Chart generated for 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC


  💾 Chart saved using GatewayDataSource: /tmp/meteora_6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN_USDC_0p10pct_bin50_chart.png
  📋 Chart stored for Telegram sending

✅ Chart generation completed for 3 pool(s)
📋 3 charts stored for Telegram sending


In [ ]:
# 📊 Generate CSV Reports with Token Symbols (Save for Later)
if all_pool_analyses:
    print("📋 Generating detailed CSV reports...\n")
    
    # Clear any existing CSV files from previous runs
    telegram_reports['csv_files'] = []
    
    for i, analysis in enumerate(all_pool_analyses, 1):
        if 'bins_df' in analysis and not analysis['bins_df'].empty:
            print(f"📄 CSV Report {i}/{len(all_pool_analyses)}: {analysis['pool_name']}")
            
            # Use cached token symbols from initial pool fetching
            base_symbol = analysis['base_token']
            quote_symbol = analysis['quote_token']
            
            # Use GatewayDataSource token cache if available and token symbols are still addresses
            if analysis.get('base_token_address') and len(base_symbol) > 10:
                cache_key = f"{analysis['base_token_address']}_{NETWORK}"
                if cache_key in gateway_ds.token_cache:
                    cached_info = gateway_ds.token_cache[cache_key]
                    if cached_info and 'symbol' in cached_info:
                        base_symbol = cached_info['symbol']
                        
            if analysis.get('quote_token_address') and len(quote_symbol) > 10:
                cache_key = f"{analysis['quote_token_address']}_{NETWORK}"
                if cache_key in gateway_ds.token_cache:
                    cached_info = gateway_ds.token_cache[cache_key]
                    if cached_info and 'symbol' in cached_info:
                        quote_symbol = cached_info['symbol']
            
            token_pair = f"{base_symbol}/{quote_symbol}"
            
            # Prepare detailed DataFrame
            detailed_df = analysis['bins_df'].copy()
            
            # Add calculated fields with proper token symbols, fee tier, and bin step
            detailed_df['token_pair'] = token_pair
            detailed_df['base_token_symbol'] = base_symbol
            detailed_df['quote_token_symbol'] = quote_symbol
            detailed_df['fee_pct'] = analysis['fee_pct']
            detailed_df['bin_step'] = analysis['bin_step']
            detailed_df['pool_name'] = analysis['pool_name']
            detailed_df['pool_address'] = analysis['address']
            detailed_df['base_token_address'] = analysis.get('base_token_address', '')
            detailed_df['quote_token_address'] = analysis.get('quote_token_address', '')
            detailed_df['current_price'] = analysis['current_price']
            detailed_df['is_active_bin'] = detailed_df['binId'] == analysis['active_bin_id']
            detailed_df['distance_from_price'] = ((detailed_df['price'] - analysis['current_price']) / analysis['current_price'] * 100)
            detailed_df['total_value_usd'] = (
                detailed_df['baseTokenAmount'] * analysis['current_price'] + 
                detailed_df['quoteTokenAmount']
            )
            detailed_df['liquidity_share_pct'] = (detailed_df['total_value_usd'] / detailed_df['total_value_usd'].sum() * 100)
            
            # Reorder columns with token information, fee tier, and bin step first
            column_order = [
                'token_pair', 'base_token_symbol', 'quote_token_symbol', 'fee_pct', 'bin_step',
                'pool_name', 'pool_address', 'base_token_address', 'quote_token_address',
                'binId', 'price', 'is_active_bin',
                'baseTokenAmount', 'quoteTokenAmount', 'total_value_usd',
                'liquidity_share_pct', 'distance_from_price', 'current_price'
            ]
            
            detailed_df = detailed_df[column_order]
            
            # Create CSV filename with token pair, fee tier, and bin step
            timestamp_str = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M")
            safe_pair_name = token_pair.replace('/', '_')
            fee_tier = f"{analysis['fee_pct']:.2f}".replace('.', 'p')  # 0.04 -> 0p04
            bin_step = analysis['bin_step']
            csv_filename = f"/tmp/meteora_{safe_pair_name}_{fee_tier}pct_bin{bin_step}_{timestamp_str}.csv"
            
            # Create metadata for GatewayDataSource export
            csv_metadata = {
                'token_pair': token_pair,
                'pool_address': analysis['address'],
                'network': NETWORK,
                'analysis_timestamp': gateway_ds.format_timestamp(analysis['timestamp'])
            }
            
            # Use GatewayDataSource export_to_csv method
            if gateway_ds.export_to_csv(detailed_df, csv_filename, metadata=csv_metadata):
                print(f"  ✅ CSV exported using GatewayDataSource: {csv_filename}")
                print(f"  📊 Records: {len(detailed_df)}")
                
                # Display sample data with token symbols, fee tier, and bin step using GatewayDataSource formatting
                print(f"  📋 CSV Preview (top 3 rows):")
                display_df = detailed_df.head(3)[['token_pair', 'fee_pct', 'bin_step', 'binId', 'price', 'total_value_usd', 'liquidity_share_pct']]
                for _, row in display_df.iterrows():
                    print(f"    {row['token_pair']} {gateway_ds.format_percentage(row['fee_pct'])} Bin{row['bin_step']} | "
                          f"Bin {row['binId']}: {gateway_ds.format_price(row['price'])} | "
                          f"{gateway_ds.format_number(row['total_value_usd'], is_currency=True)} "
                          f"({gateway_ds.format_percentage(row['liquidity_share_pct'])})")
                
                # Store CSV info for Telegram sending later with enhanced caption
                csv_info = {
                    'filename': csv_filename,
                    'pool_name': analysis['pool_name'],
                    'token_pair': token_pair,
                    'caption': f"""📋 <b>Detailed Pool Data Export</b>
🏊 Pool: {token_pair} • {gateway_ds.format_percentage(analysis['fee_pct'])} Fee • Bin:{analysis['bin_step']}
📅 {gateway_ds.format_timestamp(datetime.now(timezone.utc))}

📊 Dataset Summary:
• Total Bins: {len(detailed_df)}
• Active Bin: {analysis['active_bin_id']}
• Price Range: {gateway_ds.format_price(detailed_df['price'].min())} - {gateway_ds.format_price(detailed_df['price'].max())}
• Total Liquidity: {gateway_ds.format_number(analysis['total_liquidity_usd'], is_currency=True)}
• Fee Tier: {gateway_ds.format_percentage(analysis['fee_pct'])} | Bin Step: {analysis['bin_step']}
"""
                }
                telegram_reports['csv_files'].append(csv_info)
                print(f"  📋 CSV stored for Telegram sending")
                
            else:
                print(f"  ❌ Error exporting CSV with GatewayDataSource")
            
            print()  # Empty line between pools
            
        else:
            print(f"  ⚠️ No bin data available for {analysis['pool_name']}")
    
    print(f"✅ CSV report generation completed for all pools")
    print(f"📋 {len(telegram_reports['csv_files'])} CSV files stored for Telegram sending")
else:
    print("⚠️ No pool analyses available for CSV export")

📋 Generating detailed CSV reports...

📄 CSV Report 1/3: SOL/USDC Pool
  ✅ CSV exported using GatewayDataSource: /tmp/meteora_SOL_USDC_0p04pct_bin100_20250924_0347.csv
  📊 Records: 141
  📋 CSV Preview (top 3 rows):
    SOL/USDC 0.04% Bin100 | Bin -225: $106.58 | $158.94 (0.27%)
    SOL/USDC 0.04% Bin100 | Bin -224: $107.65 | $163.19 (0.28%)
    SOL/USDC 0.04% Bin100 | Bin -223: $108.73 | $180.27 (0.31%)
  📋 CSV stored for Telegram sending

📄 CSV Report 2/3: JUP/SOL Pool
  ✅ CSV exported using GatewayDataSource: /tmp/meteora_JUP_SOL_0p15pct_bin80_20250924_0347.csv
  📊 Records: 141
  📋 CSV Preview (top 3 rows):
    JUP/SOL 0.15% Bin80 | Bin 28: $0.0012500 | $0.00 (0.00%)
    JUP/SOL 0.15% Bin80 | Bin 29: $0.0012600 | $0.00 (0.00%)
    JUP/SOL 0.15% Bin80 | Bin 30: $0.0012700 | $0.06 (0.00%)
  📋 CSV stored for Telegram sending

📄 CSV Report 3/3: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC Pool
  ✅ CSV exported using GatewayDataSource: /tmp/meteora_6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSq

In [214]:
# 📤 Send All Reports to Telegram
print("🚀 SENDING ALL REPORTS TO TELEGRAM")
print("=" * 60)

telegram_notifier = notification_manager.get_notifier('telegram')
if not telegram_notifier:
    print("❌ Telegram notifier not configured")
    print("💡 Configure Telegram in .env to send reports")
else:
    # Summary of what will be sent
    text_count = 1 if telegram_reports['report_message'] else 0
    chart_count = len(telegram_reports['chart_files'])
    csv_count = len(telegram_reports['csv_files'])
    total_items = text_count + chart_count + csv_count
    
    print(f"📊 Sending {total_items} items to Telegram:")
    print(f"  • {text_count} text report")
    print(f"  • {chart_count} chart files")
    print(f"  • {csv_count} CSV files")
    print()
    
    sent_count = 0
    failed_count = 0
    
    # Send text report first
    if telegram_reports['report_message']:
        print("1️⃣ Sending text report...")
        try:
            notification_msg = NotificationMessage(
                title="Meteora Pool Analysis",
                message=telegram_reports['report_message'],
                level="info"
            )
            
            if await telegram_notifier.send_notification(notification_msg):
                print("  ✅ Text report sent successfully")
                sent_count += 1
            else:
                print("  ❌ Failed to send text report")
                failed_count += 1
        except Exception as e:
            print(f"  ❌ Error sending text report: {e}")
            failed_count += 1
        print()
    
    # Send chart files
    if telegram_reports['chart_files']:
        print("2️⃣ Sending chart files...")
        for i, chart_info in enumerate(telegram_reports['chart_files'], 1):
            print(f"  📊 Chart {i}/{chart_count}: {chart_info['pool_name']}")
            try:
                if await telegram_notifier.send_photo(chart_info['filename'], chart_info['caption']):
                    print(f"    ✅ Chart sent successfully")
                    sent_count += 1
                else:
                    print(f"    ❌ Failed to send chart")
                    failed_count += 1
            except Exception as e:
                print(f"    ❌ Error sending chart: {e}")
                failed_count += 1
        print()
    
    # Send CSV files
    if telegram_reports['csv_files']:
        print("3️⃣ Sending CSV files...")
        for i, csv_info in enumerate(telegram_reports['csv_files'], 1):
            print(f"  📄 CSV {i}/{csv_count}: {csv_info['pool_name']}")
            try:
                if await telegram_notifier.send_document(csv_info['filename'], csv_info['caption']):
                    print(f"    ✅ CSV sent successfully")
                    sent_count += 1
                else:
                    print(f"    ❌ Failed to send CSV")
                    failed_count += 1
            except Exception as e:
                print(f"    ❌ Error sending CSV: {e}")
                failed_count += 1
        print()
    
    # Final summary
    print("📊 TELEGRAM SENDING SUMMARY")
    print("=" * 40)
    print(f"✅ Successfully sent: {sent_count}/{total_items}")
    print(f"❌ Failed to send: {failed_count}/{total_items}")
    
    if sent_count > 0:
        print(f"\n🎉 Check your Telegram for {sent_count} new messages!")
    
    if failed_count > 0:
        print(f"\n⚠️ {failed_count} items failed to send - check error messages above")

🚀 SENDING ALL REPORTS TO TELEGRAM
📊 Sending 7 items to Telegram:
  • 1 text report
  • 3 chart files
  • 3 CSV files

1️⃣ Sending text report...
  ✅ Text report sent successfully

2️⃣ Sending chart files...
  📊 Chart 1/3: SOL/USDC Pool
    ✅ Chart sent successfully
  📊 Chart 2/3: JUP/SOL Pool
    ✅ Chart sent successfully
  📊 Chart 3/3: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC Pool
    ✅ Chart sent successfully

3️⃣ Sending CSV files...
  📄 CSV 1/3: SOL/USDC Pool
    ✅ CSV sent successfully
  📄 CSV 2/3: JUP/SOL Pool
    ✅ CSV sent successfully
  📄 CSV 3/3: 6p6xgHyF7AeE6TZkSmFsko444wqoP15icUSqi2jfGiPN/USDC Pool
    ✅ CSV sent successfully

📊 TELEGRAM SENDING SUMMARY
✅ Successfully sent: 7/7
❌ Failed to send: 0/7

🎉 Check your Telegram for 7 new messages!


In [215]:
# 🔒 Cleanup and Execution Summary
print("✅ Meteora pool analysis completed successfully\n")

# Clean up temporary files (optional)
import os
import glob

temp_files = glob.glob('/tmp/meteora_*.png') + glob.glob('/tmp/meteora_*.html') + glob.glob('/tmp/meteora_*.csv')
cleanup_count = 0

for file_path in temp_files:
    try:
        # Keep the most recent files
        file_age = datetime.now() - datetime.fromtimestamp(os.path.getctime(file_path))
        if file_age.days > 0:  # Clean files older than 1 day
            os.remove(file_path)
            cleanup_count += 1
            print(f"🗑️ Cleaned up: {os.path.basename(file_path)}")
    except Exception as e:
        print(f"⚠️ Could not remove {file_path}: {e}")

if cleanup_count > 0:
    print(f"\n🧹 Cleaned up {cleanup_count} old temporary files")

# Execution Summary
print("\n📊 EXECUTION SUMMARY")
print("=" * 50)
print(f"🌐 Data Source: Gateway API ({GATEWAY_URL})")
print(f"📡 Using: QuantsLab GatewayDataSource (Hummingbot Gateway HTTP Client)")
print(f"🔗 Network: Solana {NETWORK}")

if 'all_pool_analyses' in locals() and all_pool_analyses:
    total_liquidity = sum(a['total_liquidity_usd'] for a in all_pool_analyses)
    print(f"🏊 Pools Analyzed: {len(all_pool_analyses)}")
    print(f"💰 Total Liquidity: {gateway_ds.format_number(total_liquidity, is_currency=True)}")
    print(f"📤 Reports Sent: {'✅ Yes' if enabled_notifiers else '❌ No notifiers configured'}")
    print(f"📊 Charts Generated: ✅ Yes (via GatewayDataSource)")
    print(f"📋 CSV Exports: ✅ Yes (via GatewayDataSource)")
    
    # Display final cache statistics
    cache_stats = gateway_ds.get_cache_stats()
    print(f"🗂️ Final Cache Stats:")
    print(f"  • Token lookups cached: {cache_stats['cached_tokens']}")
    print(f"  • Failed token lookups: {cache_stats['failed_lookups']}")
    print(f"  • Total cache entries: {cache_stats['token_cache_size']}")
    
else:
    print(f"⚠️ No pools analyzed - check Gateway connection")

print("\n🎉 Meteora pool analysis complete!")
print("🔧 Powered by QuantsLab GatewayDataSource for standardized Gateway interactions")

if 'telegram' in enabled_notifiers:
    print("📱 Check your Telegram for delivered reports, charts, and data files")
else:
    print("💡 Configure Telegram in .env to receive automated reports")

✅ Meteora pool analysis completed successfully


📊 EXECUTION SUMMARY
🌐 Data Source: Gateway API (http://localhost:15888)
📡 Using: QuantsLab GatewayDataSource (Hummingbot Gateway HTTP Client)
🔗 Network: Solana mainnet-beta
🏊 Pools Analyzed: 3
💰 Total Liquidity: $317.45M
📤 Reports Sent: ✅ Yes
📊 Charts Generated: ✅ Yes (via GatewayDataSource)
📋 CSV Exports: ✅ Yes (via GatewayDataSource)
🗂️ Final Cache Stats:
  • Token lookups cached: 3
  • Failed token lookups: 1
  • Total cache entries: 4

🎉 Meteora pool analysis complete!
🔧 Powered by QuantsLab GatewayDataSource for standardized Gateway interactions
📱 Check your Telegram for delivered reports, charts, and data files
